# Data encoding (M samples of N features)

**Basis Encoding:** |X> = 1/sqrt(M) * sumi(1,M,|xi>)
+ Disadvantage: the state vectors can become quite sparse

**Amplitude Encoding:**
+ Each entry of each data point corresponds to a basis vector and the amplitude is its value/sum
+ The advantage of amplitude encoding is that it only requires log(N*M) qubits to encode 

**Angle Encoding:** tensor product of quantum encoding of each features 
+ Dense angle encoding 

**Arbitrary encoding:** encodes N features as rotations on N parameterized gates on n qubits, where n <= M

## Basis encoding

In [1]:
import math
from qiskit import QuantumCircuit

desired_state = [
    0,
    0,
    0,
    0,
    0,
    1 / math.sqrt(2),
    0,
    1 / math.sqrt(2)]

qc = QuantumCircuit(3)
qc.initialize(desired_state, [0,1,2])
qc.decompose().decompose().decompose().decompose().decompose().draw()

┌───────────────────────┐                      ┌───┐»
q_0: ─|0>─┤ multiplex1_reverse_dg ├──────────────────────┤ X ├»
          └──────┬─────────┬──────┘┌───┐┌──────────┐┌───┐└─┬─┘»
q_1: ─|0>────────┤ Ry(π/4) ├───────┤ X ├┤ Ry(-π/4) ├┤ X ├──■──»
                 ├─────────┴┐      └─┬─┘└──────────┘└─┬─┘     »
q_2: ─|0>────────┤ R(π,π/2) ├────────■────────────────■───────»
                 └──────────┘                                 »
«     ┌───────────────────────────────┐┌───┐»
«q_0: ┤ multiplex1_reverse_reverse_dg ├┤ X ├»
«     └───────────────────────────────┘└─┬─┘»
«q_1: ───────────────────────────────────┼──»
«                                        │  »
«q_2: ───────────────────────────────────■──»
«                                           »
«     ┌───────────────────────────────────────┐┌───┐»
«q_0: ┤ multiplex1_reverse_reverse_reverse_dg ├┤ X ├»
«     └───────────────────────────────────────┘└─┬─┘»
«q_1: ───────────────────────────────────────────■──»
«                                                   »
«q_2: ──────────────────────────────────────────────»
«                                                   »
«     ┌───────────────────────────────┐┌───┐
«q_0: ┤ multiplex1_reverse_reverse_dg ├┤ X ├
«     └───────────────────────────────┘└─┬─┘
«q_1: ───────────────────────────────────┼──
«                                        │  
«q_2: ───────────────────────────────────■──
«

## Amplitude encoding

In [2]:
desired_state = [
    1 / math.sqrt(15.25) * 1.5,
    0,
    1 / math.sqrt(15.25) * -2,
    1 / math.sqrt(15.25) * 3]

qc = QuantumCircuit(2)
qc.initialize(desired_state, [0,1]) 
# both basis encoding amplitude encoding used "initialize()" function
# However the desired state is very different

qc.decompose().decompose().decompose().decompose().decompose().draw()

┌─────────────┐            ┌───┐┌──────────────┐┌─────────┐┌───┐»
q_0: ─|0>──┤ Ry(0.98279) ├────────────┤ X ├┤ Ry(-0.98279) ├┤ Rz(π/2) ├┤ X ├»
          ┌┴─────────────┴┐┌─────────┐└─┬─┘└──────────────┘└─────────┘└─┬─┘»
q_1: ─|0>─┤ R(2.3531,π/2) ├┤ U1(π/2) ├──■───────────────────────────────■──»
          └───────────────┘└─────────┘                                     »
«     ┌──────────┐
«q_0: ┤ Rz(-π/2) ├
«     └──────────┘
«q_1: ────────────
«

## Angle encoding 

(encode only 1 data point at a time)

In [ ]:
qc = QuantumCircuit(3)
qc.ry(0, 0)
qc.ry(2*math.pi/4, 1)
qc.ry(2*math.pi/2, 2)
qc.draw()

## Other abstract encoding

### EfficientSU2 to encode 12 features

In [5]:
from qiskit.circuit.library import EfficientSU2
circuit = EfficientSU2(num_qubits=3, reps=1, insert_barriers=True)
print(circuit.decompose().draw())

x = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2]
encode = circuit.bind_parameters(x)
encode.decompose().draw()

     ┌──────────┐┌──────────┐ ░            ░ ┌──────────┐ ┌──────────┐
q_0: ┤ Ry(θ[0]) ├┤ Rz(θ[3]) ├─░────────■───░─┤ Ry(θ[6]) ├─┤ Rz(θ[9]) ├
     ├──────────┤├──────────┤ ░      ┌─┴─┐ ░ ├──────────┤┌┴──────────┤
q_1: ┤ Ry(θ[1]) ├┤ Rz(θ[4]) ├─░───■──┤ X ├─░─┤ Ry(θ[7]) ├┤ Rz(θ[10]) ├
     ├──────────┤├──────────┤ ░ ┌─┴─┐└───┘ ░ ├──────────┤├───────────┤
q_2: ┤ Ry(θ[2]) ├┤ Rz(θ[5]) ├─░─┤ X ├──────░─┤ Ry(θ[8]) ├┤ Rz(θ[11]) ├
     └──────────┘└──────────┘ ░ └───┘      ░ └──────────┘└───────────┘


┌─────────┐┌─────────┐ ░            ░ ┌─────────┐ ┌───────┐ 
q_0: ┤ Ry(0.1) ├┤ Rz(0.4) ├─░────────■───░─┤ Ry(0.7) ├─┤ Rz(1) ├─
     ├─────────┤├─────────┤ ░      ┌─┴─┐ ░ ├─────────┤┌┴───────┴┐
q_1: ┤ Ry(0.2) ├┤ Rz(0.5) ├─░───■──┤ X ├─░─┤ Ry(0.8) ├┤ Rz(1.1) ├
     ├─────────┤├─────────┤ ░ ┌─┴─┐└───┘ ░ ├─────────┤├─────────┤
q_2: ┤ Ry(0.3) ├┤ Rz(0.6) ├─░─┤ X ├──────░─┤ Ry(0.9) ├┤ Rz(1.2) ├
     └─────────┘└─────────┘ ░ └───┘      ░ └─────────┘└─────────┘

### ZZFeatureMap 
circuit with 3 qubits, only encodes a data point of 3 features, despite having 6 parameterized gates

In [9]:
from qiskit.circuit.library import ZZFeatureMap
circuit = ZZFeatureMap(3, reps=1, insert_barriers=True)
print(circuit.decompose().draw())

x = [0.1, 0.2, 0.3]
encode = circuit.bind_parameters(x)
encode.decompose().draw()

     ┌───┐ ░ ┌─────────────┐                                               »
q_0: ┤ H ├─░─┤ P(2.0*x[0]) ├──■────────────────────────────────────■────■──»
     ├───┤ ░ ├─────────────┤┌─┴─┐┌──────────────────────────────┐┌─┴─┐  │  »
q_1: ┤ H ├─░─┤ P(2.0*x[1]) ├┤ X ├┤ P(2.0*(π - x[0])*(π - x[1])) ├┤ X ├──┼──»
     ├───┤ ░ ├─────────────┤└───┘└──────────────────────────────┘└───┘┌─┴─┐»
q_2: ┤ H ├─░─┤ P(2.0*x[2]) ├──────────────────────────────────────────┤ X ├»
     └───┘ ░ └─────────────┘                                          └───┘»
«                                               »
«q_0: ──────────────────────────────────■───────»
«                                       │       »
«q_1: ──────────────────────────────────┼────■──»
«     ┌──────────────────────────────┐┌─┴─┐┌─┴─┐»
«q_2: ┤ P(2.0*(π - x[0])*(π - x[2])) ├┤ X ├┤ X ├»
«     └──────────────────────────────┘└───┘└───┘»
«                                          
«q_0: ─────────────────────────────────────
«                      

┌───┐ ░ ┌────────┐                                                   »
q_0: ┤ H ├─░─┤ P(0.2) ├──■─────────────────■────■─────────────────■───────»
     ├───┤ ░ ├────────┤┌─┴─┐┌───────────┐┌─┴─┐  │                 │       »
q_1: ┤ H ├─░─┤ P(0.4) ├┤ X ├┤ P(17.894) ├┤ X ├──┼─────────────────┼────■──»
     ├───┤ ░ ├────────┤└───┘└───────────┘└───┘┌─┴─┐┌───────────┐┌─┴─┐┌─┴─┐»
q_2: ┤ H ├─░─┤ P(0.6) ├───────────────────────┤ X ├┤ P(17.286) ├┤ X ├┤ X ├»
     └───┘ ░ └────────┘                       └───┘└───────────┘└───┘└───┘»
«                       
«q_0: ──────────────────
«                       
«q_1: ───────────────■──
«     ┌───────────┐┌─┴─┐
«q_2: ┤ P(16.718) ├┤ X ├
«     └───────────┘└───┘